# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [142]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [143]:
# Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
# separación de los nombres por fila [pd.explode] 
cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                       "KingsDucks": "Kings,Ducks",
                                       "Red Wings": "Red,Wings", 
                                       "Maple Leafs": "Maple,Leafs", 
                                       "Blue Jackets": "Blue,Jackets",
                                       "Golden Knights": "Golden,Knights" })
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
cities = cities.explode("NHL")
cities.head(10)

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,Rangers
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,Islanders
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,Devils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,Kings
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,Ducks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers


In [144]:
# Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por espaciado
# dejando solo el valor de los ultimos datos de cada celda del equipo
nhl_df = nhl_df[nhl_df["year"] == 2018]
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])
nhl_df.head(10)

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
6,Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL
7,Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL
8,Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL
9,Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,2018,NHL


In [146]:
# Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
# juegos [W+L]
df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
df["W-L%"] = df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
df["W-L%"] = df["W-L%"].astype("float")

df.head(10)

,Metropolitan area,Population (2016 est.)[8],NHL,team,W,L,W-L%
0,New York City,20153634.0,Rangers,Rangers,34,39,0.465753
1,New York City,20153634.0,Islanders,Islanders,35,37,0.486111
2,New York City,20153634.0,Devils,Devils,44,29,0.602740
3,Los Angeles,13310447.0,Kings,Kings,45,29,0.608108
4,Los Angeles,13310447.0,Ducks,Ducks,44,25,0.637681
5,San Francisco Bay Area,6657982.0,Sharks,Sharks,45,27,0.625000
6,Chicago,9512999.0,Blackhawks,Blackhawks,33,39,0.458333
7,Dallas–Fort Worth,7233323.0,Stars,Stars,42,32,0.567568
8,"Washington, D.C.",6131977.0,Capitals,Capitals,49,26,0.653333
9,Philadelphia,6070500.0,Flyers,Flyers,42,26,0.617647


In [17]:
# Agrupacion y conocimiento promedio de las regiones de valores de población y W-L%
mdf = df.groupby('Metropolitan area').mean()
population_by_region  = mdf.iloc[:,0]
win_loss_by_region = mdf.iloc[:,1]

# Calculo de la correlación de los valores calculados
result = stats.pearsonr(population_by_region, win_loss_by_region)
result[0]

0.012486162921209895

In [64]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
    # separación de los nombres por fila [pd.explode] 
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(","))
    cities = cities.explode("NHL")    
    # Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por espaciado
    # dejando solo el valor de los ultimos datos de cada celda del equipo
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", ""))
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    # Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
    # juegos [W+L]
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # Agrupacion y conocimiento promedio de las regiones de valores de población y W-L%
    return df.groupby('Metropolitan area').mean()

def nhl_correlation(): 
    mdf = nhl()
    population_by_region  = mdf.iloc[:,0]
    win_loss_by_region = mdf.iloc[:,1]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [65]:
nhl_correlation()

(0.012486162921209895, 0.9497182859911791)

### NOTA: 
Dado que las preguntas de la 2 - 4 es similar que la pregunta 1 cambiando la base de datos se desarrollan de forma similar, se utiliza el mismo programa diseñado para la pregunta 1 con la modificación de cada base de datos requerida 

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [67]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
def nba():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
    # separación de los nombres por fila [pd.explode] 
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                           "LakersClippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
    cities = cities.explode("NBA")    
    # Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por 
    # espaciado dejando solo el valor de los ultimos datos de cada celda del equipo
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "", x))
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])

    # Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
    # juegos [W+L]
    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")

    # Agrupacion y conocimiento promedio de las regiones de valores de población y W-L%
    return df.groupby('Metropolitan area').mean()

def nba_correlation():
    mdf = nba()
    population_by_region  = mdf.iloc[:,0]
    win_loss_by_region = mdf.iloc[:,1]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [69]:
nba_correlation()

(-0.17657160252844611, 0.3687474160446298)

In [68]:
nba()

,Population (2016 est.)[8],W-L%
Metropolitan area,,
Atlanta,5789700.0,0.292683
Boston,4794447.0,0.670732
Charlotte,2474314.0,0.439024
Chicago,9512999.0,0.329268
Cleveland,2055612.0,0.609756
Dallas–Fort Worth,7233323.0,0.292683
Denver,2853077.0,0.560976
Detroit,4297617.0,0.475610
Houston,6772470.0,0.792683


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [174]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
    # separación de los nombres por fila [pd.explode] 
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "CubsWhite Sox": "Cubs,White Sox", 
                                           "DodgersAngels": "Dodgers,Angels", 
                                           "GiantsAthletics": "Giants,Athletics", 
                                           "YankeesMets": "Yankees,Mets",
                                           "Red Sox": "Red Sox"})
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
    cities = cities.explode("MLB")    
    # Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por 
    # espaciado dejando solo el valor de los ultimos datos de cada celda del equipo
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-1])
    
    # Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
    # juegos [W+L]
    df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")
    
    # Agrupacion y conocimiento promedio de las regiones de valores de población y W-L%
    return df.groupby('Metropolitan area').mean()

def mlb_correlation(): 
    mdf = mlb()
    population_by_region  = mdf.iloc[:,0]
    win_loss_by_region = mdf.iloc[:,1]
    #raise NotImplementedError()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [180]:
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
# Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
# separación de los nombres por fila [pd.explode] 
cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x))
cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue Jays", 
                                       "CubsWhite Sox": "Cubs,White Sox", 
                                       "DodgersAngels": "Dodgers,Angels", 
                                       "GiantsAthletics": "Giants,Athletics", 
                                       "YankeesMets": "Yankees,Mets",
                                       "Red Sox": "Red Sox"})
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
cities = cities.explode("MLB")    
# Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por 
# espaciado dejando solo el valor de los ultimos datos de cada celda del equipo
mlb_df = mlb_df[mlb_df["year"] == 2018]
mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(" ")[-2])

# Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
# juegos [W+L]
df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L"]]
df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))
df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
df["W-L%"] = df["W-L%"].astype("float")


TypeError: unhashable type: 'list'

In [173]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],Yankees,KnicksNets,RangersIslandersDevils[note 3]
0,New York City,20153634,GiantsJets[note 1],Mets,KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],Dodgers,LakersClippers,KingsDucks
1,Los Angeles,13310447,RamsChargers[note 4],Angels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],Giants,Warriors,Sharks[note 7]
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs,Bulls[note 9],Blackhawks
3,Chicago,9512999,Bears[note 8],White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards[note 11],Capitals


In [179]:
mlb_df

,team,W,L,W-L%,GB,year,League
0,Red,108,54,0.667,--,2018,MLB
1,York,100,62,0.617,8.0,2018,MLB
2,Bay,90,72,0.556,18.0,2018,MLB
3,Blue,73,89,0.451,35.0,2018,MLB
4,Baltimore,47,115,0.290,61.0,2018,MLB
5,Cleveland,91,71,0.562,--,2018,MLB
6,Minnesota,78,84,0.481,13.0,2018,MLB
7,Detroit,64,98,0.395,27.0,2018,MLB
8,White,62,100,0.383,29.0,2018,MLB
9,City,58,104,0.358,33.0,2018,MLB


In [177]:
df

,Metropolitan area,Population (2016 est.)[8],MLB,team,W,L,W-L%


In [163]:
get_WLrate_Population('MLB')

,W,L,team,team_name,Region,Metropolitan area,Population,MLB
0,108,54,Boston Red Sox,"[Boston, Red, Sox]",Boston,Boston,4794447,Red Sox
1,100,62,New York Yankees,"[New, York, Yankees]",New York City,New York City,20153634,YankeesMets
2,77,85,New York Mets,"[New, York, Mets]",New York City,New York City,20153634,YankeesMets
3,90,72,Tampa Bay Rays,"[Tampa, Bay, Rays]",Tampa Bay Area,Tampa Bay Area,3032171,Rays
4,73,89,Toronto Blue Jays,"[Toronto, Blue, Jays]",Toronto,Toronto,5928040,Blue Jays
5,47,115,Baltimore Orioles,"[Baltimore, Orioles]",Baltimore,Baltimore,2798886,Orioles
6,91,71,Cleveland Indians,"[Cleveland, Indians]",Cleveland,Cleveland,2055612,Indians
7,78,84,Minnesota Twins,"[Minnesota, Twins]",Minneapolis–Saint Paul,Minneapolis–Saint Paul,3551036,Twins
8,64,98,Detroit Tigers,"[Detroit, Tigers]",Detroit,Detroit,4297617,Tigers
9,62,100,Chicago White Sox,"[Chicago, White, Sox]",Chicago,Chicago,9512999,CubsWhite Sox


In [71]:
mlb_correlation()

(0.20300647973862876, 0.3199153121370018)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [72]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Limpieza del DF de los nombres, eliminiacion del caracteres especiales, cambio de nombres con adición de comas y 
    # separación de los nombres por fila [pd.explode] 
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x))
    cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                           "RamsChargers": "Rams,Chargers",
                                           "49ersRaiders": "49ers,Raiders"})
    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
    cities = cities.explode("NFL")    
    # Limpieza del DF nhl_df, seleccionando el año de interes, eliminar los caracteres especiales, y separar por 
    # espaciado dejando solo el valor de los ultimos datos de cada celda del equipo
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "", x))
    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(" ")[-1])
    
    # Union de los DF, selección de solo las areas de interes, cálculo de la relation entre victorias ["w"] y total de 
    # juegos [W+L]
    df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L"]]
    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    df["W-L%"] = df["W-L%"].astype("float")
    
    # Agrupacion y conocimiento promedio de las regiones de valores de población y W-L%
    return df.groupby('Metropolitan area').mean()

def nfl_correlation(): 
    mdf = nfl()
    population_by_region  = mdf.iloc[:,0]
    win_loss_by_region = mdf.iloc[:,1]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [75]:
nfl_correlation()

(0.004922112149349428, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [115]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=mlb()
nhl_df=nhl()
nba_df=nba()
nfl_df=nfl()

def sports_team_performance():
    mlb_df=mlb()
    nhl_df=nhl()
    nba_df=nba()
    nfl_df=nfl()
    sportsname = ['MLB', 'NHL', 'NBA', 'NFL']
    sports = [mlb_df,nhl_df,nba_df,nfl_df]
    
    p_values = pd.DataFrame({k:np.nan for k in sportsname}, index=sportsname)
    
    for i,si in enumerate(sportsname):
        for j,sj in enumerate(sportsname):
            if i!=j:
                Mi=pd.DataFrame(sports[i])
                Mj=pd.DataFrame(sports[j])
                Mi=Mi['W-L%']
                Mj=Mj['W-L%']
                merge=pd.merge(Mi,Mj,how='inner',left_index=True,right_index=True)
                #print(merge)
                p_values.loc[si,sj]=stats.ttest_rel(merge['W-L%_x'],merge['W-L%_y'])[1]

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan

    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [116]:
sports_team_performance()

AssertionError: The MLB-NFL p-value should be around 0.80

In [114]:
sports_team_performance()

,MLB,NHL,NBA,NFL
MLB,NaN,0.001097,0.861549,0.736544
NHL,0.001097,NaN,0.022297,0.030883
NBA,0.861549,0.022297,NaN,0.941792
NFL,0.736544,0.030883,0.941792,NaN


In [80]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=mlb()
nhl_df=nhl()
nba_df=nba()
nfl_df=nfl()
sportsname = ['MLB', 'NHL', 'NBA', 'NFL']
sports = [mlb_df,nhl_df,nba_df,nfl_df]

In [108]:
p_values = pd.DataFrame({k:np.nan for k in sportsname}, index=sportsname)
p_values

,MLB,NHL,NBA,NFL
MLB,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NFL,NaN,NaN,NaN,NaN


In [109]:
for i,si in enumerate(sportsname):
    for j,sj in enumerate(sportsname):
        if i!=j:
            Mi=pd.DataFrame(sports[i])
            Mj=pd.DataFrame(sports[j])
            Mi=Mi['W-L%']
            Mj=Mj['W-L%']
            merge=pd.merge(Mi,Mj,how='inner',left_index=True,right_index=True)
            #print(merge)
            p_values.loc[si,sj]=stats.ttest_rel(merge['W-L%_x'],merge['W-L%_y'])[1]

In [110]:
p_values

,MLB,NHL,NBA,NFL
MLB,NaN,0.001097,0.861549,0.736544
NHL,0.001097,NaN,0.022297,0.030883
NBA,0.861549,0.022297,NaN,0.941792
NFL,0.736544,0.030883,0.941792,NaN


In [79]:
import pandas as pd
import numpy as np
import scipy.stats as stats
#import rehttps://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i!=j:
                Mi=get_WLrate_Population(i)
                Mj=get_WLrate_Population(j)
                Mi=Mi['W/L rate']
                Mj=Mj['W/L rate']
                merge=pd.merge(Mi,Mj,how='inner',left_index=True,right_index=True)
                #print(merge)
                p_values.loc[i, j]=stats.ttest_rel(merge['W/L rate_x'],merge['W/L rate_y'])[1]
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN


In [162]:
def get_WLrate_Population(Big4):
    #Big4
    
    #html
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #1--
    #
    cities.rename(columns={'Population (2016 est.)[8]':'Population'},inplace=True)
    
    #
    cities.replace(to_replace='\[[\w ]*\]$',value='',regex=True,inplace=True)
    
    #NHL
    cities=cities[['Metropolitan area','Population',Big4]]
    
    #'-'
    cities.replace(to_replace='',value=np.nan,regex=True,inplace=True)
    cities.replace(to_replace='',value=np.nan,regex=True,inplace=True)
    cities.dropna(inplace=True)
    
    #indexindex
    cities.reset_index(inplace=True)
    cities.drop('index',axis=1,inplace=True)
    
    #_list[,]
    pattern='([A-Z]{0,2}[a-z0-9]+ [A-Z]{0,2}[a-z]+|[A-Z]{0,2}[a-z0-9]+)'
    cty_team=[]
    for i in range(len(cities)):
        a=re.findall(pattern,cities.iloc[i,2])
        #print(a,len(a))
        for j in a:
            cty_team.append([cities.iloc[i,0],j])
            
    #        
    for i in cty_team:
        i[1]=re.split(' ',i[1])
    #print(len(cty_team))
    
    #2-- 
    
    #18  
    big4_df=pd.read_csv('assets/'+Big4.lower()+'.csv')
    big4_df2018=big4_df[big4_df['year']==2018]
    
    #'W''L''team'
    big4_df2018=big4_df2018[['W','L','team']]
    
    #*?'team_name'
    #'*$'
    big4_df2018['team'].replace(to_replace='\([0-9]{0,2}\)$',value='',regex=True,inplace=True)
    big4_df2018['team'].replace(to_replace='[\*+]?[\s]?$',value='',regex=True,inplace=True)

    #team
    lists=[]
    for i in range(len(big4_df2018['team'])):
        lists.append(re.split(' ',big4_df2018['team'][i]))
    #print(lists)
    
    #team_name
    big4_df2018['team_name']=lists
    #big4_df2018['team_name']=big4_df2018['team'].str.extract('([\w]*$)')

    #
    #cty_team12
    def get_region(item):
        for i in cty_team:
            if len(i[-1])==1 and i[-1][-1]==item[-1] :
                return i[0]
            elif len(i[-1])==2 and i[-1][-1]==item[-1] and i[-1][-2]==item[-2]:
                return i[0]
    big4_df2018['Region']=big4_df2018['team_name'].apply(get_region)

    # #
    merge1=pd.merge(big4_df2018,cities,left_on='Region',right_on='Metropolitan area')
    #int
    merge1['W']=merge1['W'].astype('int')
    merge1['L']=merge1['L'].astype('int')
    
    #
    merge2=merge1.groupby('Region').agg({'W':np.sum,'L':np.sum,'Population':np.unique})
    merge2['W/L rate']=merge2['W']/(merge2['L']+merge2['W'])
    merge2['Population']=merge2['Population'].astype('int')
    
    #DataFrame
    return merge2